# DATA

In [1]:
# impor libraries
import pandas as pd
import numpy as np

In [2]:
#reading files and adding colums
col_names = ['user1', 'user2', 'Time']
#elimina nrow
file1 = pd.read_csv(r"data/sx-stackoverflow-a2q.txt", header=None,names=col_names,sep=" ")
file2 = pd.read_csv(r"data/sx-stackoverflow-c2a.txt", header=None,names=col_names,sep=" ")
file3 = pd.read_csv(r"data/sx-stackoverflow-c2q.txt", header=None,names=col_names,sep=" ")

## convert Time in reasonable format

In [3]:
def make_time_stamp(day,month,year):
    '''
    INPUT: day, month and year
    OUTPUT: integer that stands for our day
    '''
    if month==2:
        time_stamp=day+month*28+year*365
    if month in [4,6,9,11]:
        time_stamp=day+month*30+year*365
    else:
        time_stamp=day+month*31+year*365
        
    return (time_stamp)

In [4]:

def conversion_time(Time_col):
    '''
    INPUT: Time column in seconds from epoch= 1970
    OUTPUT: years,months,days derived from seconds 
    '''
    time_stamp=[]
    seconds_in_day = 60 * 60 * 24
    
    for i in range(len(Time_col)):
        seconds=Time_col[i]

        #int number of days from 1970
        day=seconds // seconds_in_day

        years=(day // 365)
        months= (day - years *365) // 30
        days= (day - years * 365 - months*30) 
        time_stamp.append(make_time_stamp(days,months,years))
        

    return(time_stamp)

### file1

In [7]:
#encode answers with 10

score=[10 for i in range(len(file1.user1))]
file1['score']=score


In [8]:
file1.head(10)

,user1,user2,Time,score
0,9,8,1217567877,10
1,1,1,1217573801,10
2,13,1,1217606247,10
3,17,1,1217617639,10
4,48,2,1217618182,10
5,17,1,1217618239,10
6,19,9,1217618357,10
7,13,23,1217618560,10
8,13,11,1217618799,10
9,23,23,1217619360,10


### file2

In [9]:
file2.head(5)

,user1,user2,Time
0,1,91,1220713630
1,3,91,1220713792
2,380,350,1220715736
3,4642,2257,1220734307
4,4642,1324220,1220735746


In [11]:
#encoded the comments to question  with 5
score=[5 for i in range(len(file2.user1))]
file2['score']=score


In [12]:
file2.head(5)

,user1,user2,Time,score
0,1,91,1220713630,5
1,3,91,1220713792,5
2,380,350,1220715736,5
3,4642,2257,1220734307,5
4,4642,1324220,1220735746,5


### file3

In [13]:
file3.head(5)

,user1,user2,Time
0,4550,4550,1220729190
1,242,184,1220733503
2,4213,4946,1220768149
3,91,91,1220768295
4,2658,1874,1220771891


In [15]:
#encoded the comments to answer  with 2
score=[2 for i in range(len(file3.user1))]
file3['score']=score

In [16]:
file3.head(5)

,user1,user2,Time,score
0,4550,4550,1220729190,2
1,242,184,1220733503,2
2,4213,4946,1220768149,2
3,91,91,1220768295,2
4,2658,1874,1220771891,2


### Merge file1, file2 and file 3 in one file

In [17]:
file=pd.concat([file1,file2,file3])

In [18]:
file = file.reset_index(drop=True)

Now we create the encoded variable for time 

In [19]:
time_stamp=conversion_time(file.Time)

In [22]:
file['time_stamp']=time_stamp

In [23]:
#give a look to final file properties
file.shape

(63497050, 5)

In [24]:
file.iloc[0::10000000]

,user1,user2,Time,score,time_stamp
0,9,8,1217567877,10,14099
10000000,1631379,2661128,1380492209,10,15977
20000000,221509,221509,1284139018,5,14870
30000000,2170392,2170392,1370681087,5,15869
40000000,190597,190597,1436234662,5,16623
50000000,497718,1056118,1367201675,2,15824
60000000,19679,3792038,1436399970,2,16624


In [25]:
file.head(5)

,user1,user2,Time,score,time_stamp
0,9,8,1217567877,10,14099
1,1,1,1217573801,10,14099
2,13,1,1217606247,10,14099
3,17,1,1217617639,10,14099
4,48,2,1217618182,10,14099


In [26]:
#drop the time column since we have an encoded value which is timestamp
file.drop('Time',inplace=True,axis=1)


#### This is our dataframe: starting from this we want to construct a dictionary

In [27]:
file.head(10)

,user1,user2,score,time_stamp
0,9,8,10,14099
1,1,1,10,14099
2,13,1,10,14099
3,17,1,10,14099
4,48,2,10,14099
5,17,1,10,14099
6,19,9,10,14099
7,13,23,10,14099
8,13,11,10,14099
9,23,23,10,14099


### Create a dictionary with the following logic
* key= (userA)
* value=[(user2,time_stamp,score_associated_with_interaction),...,(..)] 

In [28]:
# inizialization
interactions={}

In [239]:
#another dictionary
from tqdm import tqdm

for i in tqdm(range(file.shape[0])):
    try:
        interactions[int(file.user1[i])].append( ( int(file.user2[i]),int(file.time_stamp[i]),int(file.score[i]) ) )
    except:
        interactions[int(file.user1[i])]=[ ( int(file.user2[i]),int(file.time_stamp[i]),int(file.score[i]) ) ]

100%|██████████| 63497050/63497050 [35:59<00:00, 29402.54it/s]


In [242]:
len(interactions)

2226243

In order to overcome the problem associated with the manipolation and the creation of this data-structure, we produce a pickle file and we retrieve it in few seconds before starting our functionalities

In [243]:
#save dictionary as pickle

import pickle


with open('data/interactions.pickle', 'wb') as handle:
    pickle.dump(interactions, handle, protocol=pickle.HIGHEST_PROTOCOL)


